In [1]:
import re
import codecs
import string
import operator
import functools
import glob
import os
import matplotlib.colors as mc
import nltk
import numpy as np
import textstat
import unicodedata
import pandas as pd
import math
import textstat
import numpy as np
import fnmatch

#nltk.install("all")

from lexicalrichness import LexicalRichness
from collections import Counter

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag, map_tag
from nltk.tokenize import sent_tokenize

from seg import Segmentation

 # Feature Selection
from sklearn.feature_selection import VarianceThreshold

# os.remove(path)
# if os.path.exists(path):
#    os.remove(path)

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Sat Oct 10 19:16:37 2020

@author: pablonicolasr
"""
from dataset import generate
from cluster import cluster_ip
from lxml import etree


actual_directory = os.getcwd()

ruta = os.path.join(actual_directory, "mini_corpus")
ruta_del_directorio = str(ruta) + "\\*.txt"
# Me produce una lista con los nombres de los archivos txt del corpus
textos_originales = glob.glob(ruta_del_directorio)


if not os.path.exists(str(actual_directory)+"\\cleaned_corpus"):
    os.mkdir(str(actual_directory)+'\\cleaned_corpus')
    
cleaned_corpus = str(actual_directory)+"\\cleaned_corpus"

# De esta manera le voy robo el nombre a los xml
path_xml = str(actual_directory) + "\\mini_corpus" + "\\*.xml"
xml_docs = glob.glob(path_xml)


if not os.path.exists(str(actual_directory)+"\\sospechoso"):
    os.mkdir(str(actual_directory)+'\\sospechoso')
else:
    print("Se elimino carpeta vieja de fragmentos sospechosos")
    patron_inicio = 'sosp*'
    patron_final = '*.txt'
    archivos = os.listdir(str(actual_directory)+"\\sospechoso")
    for archivo in archivos:
        if fnmatch.fnmatch(archivo, patron_inicio) and fnmatch.fnmatch(archivo, patron_final):
            ruta_completa = os.path.join(str(actual_directory)+"\\sospechoso", archivo)
            os.remove(ruta_completa)
            print(f"Eliminado: {ruta_completa}")

    
if not os.path.exists(str(actual_directory) + "\\xml\\output_xml"):
    os.mkdir(str(actual_directory) + "\\xml\\output_xml")
else:
    print("Se elimino carpeta vieja de fragmentos sospechosos en formato xml")
    patron_final = '*.xml'
    archivos = os.listdir(str(actual_directory) + "\\xml\\output_xml")
    for archivo in archivos:
        if fnmatch.fnmatch(archivo, patron_final):
            ruta_completa = os.path.join(str(actual_directory) + "\\xml\\output_xml", archivo)
            os.remove(ruta_completa)
            print(f"Eliminado: {ruta_completa}")


sospechosos_path = str(actual_directory) + "\\sospechoso\\"
output_xml = str(actual_directory) + "\\xml\\output_xml\\"


f = {}
# Comienzo a leer el conjunto de textos
for i in range(len(textos_originales)):
    print("File: " + str(i+1))
    f[i] = open(textos_originales[i], encoding= "utf-8-sig")
    text = f[i].read()
    f[i].close()            
    
    # Nombre de Archivo
    filename = os.path.basename(textos_originales[i])
    # Separo nombre de archivo y extension
    filename2, extension = os.path.splitext(filename)
    
    Seg = Segmentation(text)
    # La variable data_text posee el texto completo, separado en párrafos
    data_text = Seg.paraSegmentation()
    data_list = []
    length = []
    
    # Se tiene guardar el texto sin limpiar
    for segmento in data_text:
        # A cada segmento se le borra el salto de línea
        segmento = re.sub("\n", " ", segmento)
        # Se guarda la longitud del segmento
        longitud = len(segmento)
        length.append(longitud)
        # Se guarda el segmento
        data_list.append(segmento)
    
    # De aqui empezar a procesar la limpieza de texto 
    # Some code...
    # .....
    # .....
    
    rootPath = ruta + "\\"+ filename
    outputPath = str(actual_directory) + "\\csv\\"+ filename2 + ".csv"
    
    if os.path.exists(str(actual_directory) + "\\csv\\"+ filename2 + ".csv"):
        os.remove(str(actual_directory) + "\\csv\\"+ filename2 + ".csv")
    
    generate(rootPath, outputPath, filename, "english")
    
    df = pd.read_csv(outputPath)
    
    # Jugar con estos números // Tarea
    
    # samples = len(length) * 0.075
    
    # decimal, entero = math.modf(samples)
    
    # if decimal >= 0.5:
        # min_samples = entero + 1
    # else:
        # min_samples = entero
    
    # Este número fue sacado de un paper
    min_samples = 4
    
    # Normalizar de forma correcta y eficiente el indice de facilidad de lectura
    df["fleshReadingEase"] = df["fleshReadingEase"] / df["fleshReadingEase"].max()
    
    # Seleccionar los valores numéricos del df
    # Este paso se realiza para ir viendo que genera cada párrafo del txt y como fue convertido a un vector
    dataframe = df.iloc[:,[2,3,4]]
    
    # Aprendizaje No Supervisado
    cluster = cluster_ip(dataframe, min_samples, "cosine")
    labels = cluster.labels_    
    df["labels"] = labels
    
    realClusterNum=len(set(labels)) - (1 if -1 in labels else 0)
    clusterNum=len(set(labels))
    n_noise = list(labels).count(-1) 
    
    print("Estimated number of clusters: %d" % realClusterNum)
    print("Estimated number of noise points: %d" % n_noise)
    counts = df["labels"].value_counts().to_dict()
    print(counts)
    
    # Me sirve para guardar los casos de plagio y compararlos
    xml_name = os.path.basename(xml_docs[i])
    xml_name2, extension = os.path.splitext(xml_name)
    
    # Se crea el archivo txt donde se guardaran los fragmentos sospechosos
    sherlock = codecs.open(sospechosos_path + "sosp" + filename,"w","utf-8")
    
    # Los fragmentos que fueron clasificados como plagio
    # Se guardan en el archivo .txt
    # Tener en cuenta que el dataframe y la lista data_list, manejan los mismos índices
    # Por eso, si en el dataframe se encuentra un "-1" es porque este es un outlier
    for x in range(len(df["labels"])):
        l = df["labels"][x]
        if l == -1:
            print(data_list[x])
            print(x)
            sherlock.write(data_list[x]+os.linesep)       
               
    sherlock.close()    
    
    document_el = etree.Element("document", reference = xml_name2 + ".txt")
    offset = 0
    
    for y in range(len(df["labels"])):
        
        l = df["labels"][y]
        
        if l == -1: 
            
            feature_el = etree.SubElement(document_el, "feature", name="detected-plagiarism", this_offset=str(offset), this_length=str(length[y]))
    
        offset = length[y] + 1
                          
            
    et = etree.ElementTree(document_el)
    with open(output_xml+xml_name, "wb") as fe:
        et.write(fe, encoding="utf-8", xml_declaration=None, pretty_print=True)

Se elimino carpeta vieja de fragmentos sospechosos
Eliminado: C:\Users\pablonicolasr\Desktop\pablonicolas\mentodatos\plagiarism\practico3\sospechoso\sospsuspicious-document00001.txt
Eliminado: C:\Users\pablonicolasr\Desktop\pablonicolas\mentodatos\plagiarism\practico3\sospechoso\sospsuspicious-document00002.txt
Eliminado: C:\Users\pablonicolasr\Desktop\pablonicolas\mentodatos\plagiarism\practico3\sospechoso\sospsuspicious-document00003.txt
Eliminado: C:\Users\pablonicolasr\Desktop\pablonicolas\mentodatos\plagiarism\practico3\sospechoso\sospsuspicious-document00004.txt
Eliminado: C:\Users\pablonicolasr\Desktop\pablonicolas\mentodatos\plagiarism\practico3\sospechoso\sospsuspicious-document00005.txt
Eliminado: C:\Users\pablonicolasr\Desktop\pablonicolas\mentodatos\plagiarism\practico3\sospechoso\sospsuspicious-document00006.txt
Eliminado: C:\Users\pablonicolasr\Desktop\pablonicolas\mentodatos\plagiarism\practico3\sospechoso\sospsuspicious-document00007.txt
Eliminado: C:\Users\pablonicolas

Row: [47, 'suspicious-document00002.txt', 61.8, 0.13670886075949368, 0.5497076023391813]
Row: [48, 'suspicious-document00002.txt', 66.13, 0.14414414414414414, 0.6979166666666666]
Row: [49, 'suspicious-document00002.txt', 57.95, 0.07692307692307693, 0.6666666666666666]
Row: [50, 'suspicious-document00002.txt', 59.94, 0.13649851632047477, 0.5694915254237288]
Row: [51, 'suspicious-document00002.txt', 63.9, 0.1258741258741259, 0.6850393700787402]
Row: [52, 'suspicious-document00002.txt', 60.58, 0.16149068322981366, 0.7142857142857143]
Row: [53, 'suspicious-document00002.txt', 60.65, 0.16417910447761194, 0.8070175438596491]
Row: [54, 'suspicious-document00002.txt', 33.41, 0.11764705882352941, 0.7466666666666667]
Row: [55, 'suspicious-document00002.txt', 77.91, 0.6666666666666666, 1.0]
Row: [56, 'suspicious-document00002.txt', 93.81, 0.4, 1.0]
Row: [57, 'suspicious-document00002.txt', 77.91, 0.3333333333333333, 1.0]
Row: [58, 'suspicious-document00002.txt', 35.61, 0.3333333333333333, 1.0]
Ro

Row: [174, 'suspicious-document00002.txt', 33.28, 0.1556420233463035, 0.5825688073394495]
Row: [175, 'suspicious-document00002.txt', 58.76, 0.14619883040935672, 0.6174496644295302]
Row: [176, 'suspicious-document00002.txt', 67.69, 0.16165413533834586, 0.5892857142857143]
Row: [177, 'suspicious-document00002.txt', 65.66, 0.19063545150501673, 0.6296296296296297]
Row: [178, 'suspicious-document00002.txt', 64.14, 0.1175115207373272, 0.4623376623376623]
Row: [179, 'suspicious-document00002.txt', 32.23, 0.12962962962962962, 0.5957446808510638]
Row: [180, 'suspicious-document00002.txt', 38.83, 0.07954545454545454, 0.7530864197530864]
Row: [181, 'suspicious-document00002.txt', 45.43, 0.15126050420168066, 0.7029702970297029]
Row: [182, 'suspicious-document00002.txt', 18.87, 0.12179487179487179, 0.6423357664233577]
Row: [183, 'suspicious-document00002.txt', 21.4, 0.23076923076923078, 0.7317073170731707]
Row: [184, 'suspicious-document00002.txt', 59.84, 0.1270718232044199, 0.6708860759493671]
Row

Row: [321, 'suspicious-document00002.txt', 51.52, 0.125, 0.9642857142857143]
Row: [322, 'suspicious-document00002.txt', 76.72, 0.23333333333333334, 0.9565217391304348]
Row: [323, 'suspicious-document00002.txt', 44.11, 0.12142857142857143, 0.5748987854251012]
Row: [324, 'suspicious-document00002.txt', 49.96, 0.12396694214876033, 0.7009345794392523]
Row: [325, 'suspicious-document00002.txt', -15.48, 0.09615384615384616, 0.723404255319149]
Row: [326, 'suspicious-document00002.txt', 70.13, 0.07692307692307693, 0.7222222222222222]
Row: [327, 'suspicious-document00002.txt', 27.83, 0.14950166112956811, 0.627906976744186]
Row: [328, 'suspicious-document00002.txt', 12.27, 0.12790697674418605, 0.68]
Row: [329, 'suspicious-document00002.txt', 77.91, 0.3333333333333333, 1.0]
Row: [330, 'suspicious-document00002.txt', 49.48, 0.16666666666666666, 1.0]
Row: [331, 'suspicious-document00002.txt', 79.94, 0.07407407407407407, 0.92]
Row: [332, 'suspicious-document00002.txt', 59.97, 0.25, 1.0]
Row: [333, '

Row: [4, 'suspicious-document00003.txt', 59.98, 0.06666666666666667, 0.7678571428571429]
Row: [5, 'suspicious-document00003.txt', 60.32, 0.05263157894736842, 0.8055555555555556]
Row: [6, 'suspicious-document00003.txt', 76.45, 0.17391304347826086, 0.553072625698324]
Row: [7, 'suspicious-document00003.txt', 51.52, 0.125, 0.5874125874125874]
Row: [8, 'suspicious-document00003.txt', 38.99, 0.0, 1.0]
Row: [9, 'suspicious-document00003.txt', 53.65, 0.12121212121212122, 0.6111111111111112]
Row: [10, 'suspicious-document00003.txt', 68.44, 0.0967741935483871, 0.6964285714285714]
Row: [11, 'suspicious-document00003.txt', 63.53, 0.15028901734104047, 0.673469387755102]
Row: [12, 'suspicious-document00003.txt', 68.2, 0.105, 0.6815642458100558]
Row: [13, 'suspicious-document00003.txt', 104.64, 0.2727272727272727, 1.0]
Row: [14, 'suspicious-document00003.txt', 63.53, 0.1436950146627566, 0.6060606060606061]
Row: [15, 'suspicious-document00003.txt', 52.26, 0.08717948717948718, 0.574585635359116]
Row: [

...

Row: [210, 'suspicious-document00010.txt', 77.67, 0.1413427561837456, 0.6032388663967612]
Row: [211, 'suspicious-document00010.txt', 44.75, 0.15, 1.0]
Row: [212, 'suspicious-document00010.txt', 55.54, 0.08955223880597014, 0.6721311475409836]
Row: [213, 'suspicious-document00010.txt', 10.58, 0.15492957746478872, 0.7166666666666667]
Row: [214, 'suspicious-document00010.txt', 61.8, 0.12162162162162163, 0.6641221374045801]
Row: [215, 'suspicious-document00010.txt', 38.46, 0.10622710622710622, 0.5101626016260162]
Row: [216, 'suspicious-document00010.txt', 67.42, 0.10309278350515463, 0.7241379310344828]
Row: [217, 'suspicious-document00010.txt', 48.3, 0.1348314606741573, 0.8]
Row: [218, 'suspicious-document00010.txt', 66.1, 0.15, 0.6363636363636364]
Row: [219, 'suspicious-document00010.txt', 67.22, 0.1683673469387755, 0.5050100200400801]
Row: [220, 'suspicious-document00010.txt', 68.4, 0.16428571428571428, 0.6949152542372882]
Row: [221, 'suspicious-document00010.txt', 71.07, 0.13103448275862

Row: [349, 'suspicious-document00010.txt', 57.78, 0.17391304347826086, 0.8441558441558441]
Row: [350, 'suspicious-document00010.txt', 77.91, 0.3333333333333333, 1.0]
Row: [351, 'suspicious-document00010.txt', 116.15, 0.14285714285714285, 1.0]
Row: [352, 'suspicious-document00010.txt', 49.93, 0.13994910941475827, 0.5571847507331378]
Row: [353, 'suspicious-document00010.txt', 33.59, 0.07954545454545454, 0.6341463414634146]
Row: [354, 'suspicious-document00010.txt', 80.14, 0.16619718309859155, 0.5714285714285714]
Row: [355, 'suspicious-document00010.txt', 65.49, 0.1178343949044586, 0.5503597122302158]
Row: [356, 'suspicious-document00010.txt', 69.79, 0.13333333333333333, 0.7238095238095238]
Row: [357, 'suspicious-document00010.txt', 35.62, 0.08771929824561403, 0.7307692307692307]
Row: [358, 'suspicious-document00010.txt', 77.91, 0.3333333333333333, 1.0]
Row: [359, 'suspicious-document00010.txt', 56.93, 0.14285714285714285, 1.0]
Row: [360, 'suspicious-document00010.txt', 69.15, 0.108695652

Row: [499, 'suspicious-document00010.txt', 40.35, 0.13636363636363635, 0.717948717948718]
Row: [500, 'suspicious-document00010.txt', 51.52, 0.13846153846153847, 0.7410714285714286]
Row: [501, 'suspicious-document00010.txt', 31.22, 0.10697674418604651, 0.6041666666666666]
Row: [502, 'suspicious-document00010.txt', 59.67, 0.09574468085106383, 0.7529411764705882]
Row: [503, 'suspicious-document00010.txt', 77.91, 0.3333333333333333, 1.0]
Row: [504, 'suspicious-document00010.txt', 49.48, 0.16666666666666666, 1.0]
Row: [505, 'suspicious-document00010.txt', 49.11, 0.13857677902621723, 0.592274678111588]
Row: [506, 'suspicious-document00010.txt', 55.0, 0.14661654135338345, 0.5844155844155844]
Row: [507, 'suspicious-document00010.txt', 79.8, 0.19753086419753085, 0.746268656716418]
Row: [508, 'suspicious-document00010.txt', 81.29, 0.4, 1.0]
Row: [509, 'suspicious-document00010.txt', 91.78, 0.3076923076923077, 1.0]
Row: [510, 'suspicious-document00010.txt', 58.66, 0.08333333333333333, 0.75]
Row: 

Row: [655, 'suspicious-document00010.txt', 58.15, 0.07453416149068323, 0.6510067114093959]
Row: [656, 'suspicious-document00010.txt', 49.28, 0.14772727272727273, 0.695364238410596]
Row: [657, 'suspicious-document00010.txt', 75.47, 0.12251655629139073, 0.48120300751879697]
Row: [658, 'suspicious-document00010.txt', 27.33, 0.11612903225806452, 0.656934306569343]
Row: [659, 'suspicious-document00010.txt', 64.78, 0.11538461538461539, 0.5928853754940712]
Row: [660, 'suspicious-document00010.txt', 77.91, 0.3333333333333333, 1.0]
Row: [661, 'suspicious-document00010.txt', 81.29, 0.125, 0.7142857142857143]
Row: [662, 'suspicious-document00010.txt', 68.94, 0.11827956989247312, 0.6144578313253012]
Row: [663, 'suspicious-document00010.txt', 58.86, 0.12931034482758622, 0.5686274509803921]
Row: [664, 'suspicious-document00010.txt', 64.38, 0.07058823529411765, 0.55]
Row: [665, 'suspicious-document00010.txt', 65.19, 0.12322274881516587, 0.5531914893617021]
Row: [666, 'suspicious-document00010.txt', 7

Row: [824, 'suspicious-document00010.txt', 57.27, 0.10344827586206896, 0.6666666666666666]
Row: [825, 'suspicious-document00010.txt', 67.38, 0.13793103448275862, 0.6008771929824561]
Row: [826, 'suspicious-document00010.txt', 70.13, 0.14285714285714285, 0.7714285714285715]
Row: [827, 'suspicious-document00010.txt', 70.13, 0.1111111111111111, 0.7746478873239436]

***End generating***
Estimated number of clusters: 1
Estimated number of noise points: 5
{0: 822, -1: 5}
The following meditations will probably rank high among many similar works which the contemplative love of Jesus has produced; but it is our duty here plainly to affirm that they have no pretensions whatever to be regarded as history.1 They are but intended to take one of the lowest places among those numerous representations of the Passion which have been given us by pious writers and artists, and to be considered at the very utmost as the Lenten meditations of a devout nun, related in all simplicity, and written down in the